In [1]:

from dotenv import load_dotenv
load_dotenv()
from os import environ
from tqdm import tqdm

if environ.get('OPENAI_API_KEY') is not None:
    print("Using OpenAI")
else:
    from sentence_transformers import SentenceTransformer

from pymilvus import MilvusClient


embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
milvus_client = MilvusClient(uri="./milvus_demo.db")
collection_name = "my_rag_collection"

/media/thanhnv154te/15a388e9-5b07-4e1c-b175-72e9839b5c872/workspace/rag-projects/.venv/lib/python3.12/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [2]:
# Drop already exists collection
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

In [3]:
def embed_text(embedding_model, text):
    """
    Generate embedding text of the given text
    """
    return embedding_model.encode(text).tolist()

sample_text = "this is a sample sentence"
embedding = embed_text(embedding_model, sample_text)
embedding_shape = len(embedding)
print(embedding_shape)

/media/thanhnv154te/15a388e9-5b07-4e1c-b175-72e9839b5c872/workspace/rag-projects/.venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


384


In [4]:
# Create collection
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_shape,
    metric_type="IP",
    consistency_level="Bounded"
)

In [5]:
from glob import glob

text_lines = []

for file_path in glob("milvus_docs/en/faq/*.md", recursive=True):
    with open(file_path, "r") as file:
        file_text = file.read()

    text_lines += file_text.split("# ")
    
data = []
for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": embed_text(embedding_model, line), "text": line})

milvus_client.insert(collection_name=collection_name, data=data) 

Creating embeddings: 100%|██████████| 72/72 [00:00<00:00, 191.04it/s]


{'insert_count': 72, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71], 'cost': 0}

In [6]:
question = "How is data stored in milvus?" 

search_res = milvus_client.search(
    collection_name=collection_name,
    data=[
        embed_text(embedding_model, question)
    ],  # Use the `emb_text` function to convert the question to an embedding vector
    limit=3,  # Return top 3 results
    search_params={"metric_type": "IP", "params": {}},  # Inner product distance
    output_fields=["text"],  # Return the text field
)

In [7]:
search_res

data: [[{'id': 44, 'distance': 0.648801863193512, 'entity': {'text': ' Where does Milvus store data?\n\nMilvus deals with two types of data, inserted data and metadata. \n\nInserted data, including vector data, scalar data, and collection-specific schema, are stored in persistent storage as incremental log. Milvus supports multiple object storage backends, including [MinIO](https://min.io/), [AWS S3](https://aws.amazon.com/s3/?nc1=h_ls), [Google Cloud Storage](https://cloud.google.com/storage?hl=en#object-storage-for-companies-of-all-sizes) (GCS), [Azure Blob Storage](https://azure.microsoft.com/en-us/products/storage/blobs), [Alibaba Cloud OSS](https://www.alibabacloud.com/product/object-storage-service), and [Tencent Cloud Object Storage](https://www.tencentcloud.com/products/cos) (COS).\n\nMetadata are generated within Milvus. Each Milvus module has its own metadata that are stored in etcd.\n\n###'}}, {'id': 55, 'distance': 0.5974205732345581, 'entity': {'text': "How does Milvus flu

In [8]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

[
    [
        " Where does Milvus store data?\n\nMilvus deals with two types of data, inserted data and metadata. \n\nInserted data, including vector data, scalar data, and collection-specific schema, are stored in persistent storage as incremental log. Milvus supports multiple object storage backends, including [MinIO](https://min.io/), [AWS S3](https://aws.amazon.com/s3/?nc1=h_ls), [Google Cloud Storage](https://cloud.google.com/storage?hl=en#object-storage-for-companies-of-all-sizes) (GCS), [Azure Blob Storage](https://azure.microsoft.com/en-us/products/storage/blobs), [Alibaba Cloud OSS](https://www.alibabacloud.com/product/object-storage-service), and [Tencent Cloud Object Storage](https://www.tencentcloud.com/products/cos) (COS).\n\nMetadata are generated within Milvus. Each Milvus module has its own metadata that are stored in etcd.\n\n###",
        0.648801863193512
    ],
    [
        "How does Milvus flush data?\n\nMilvus returns success when inserted data are loaded to th

In [9]:
# Use llm to get response

context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

SYSTEM_PROMPT = """
Human: You are an AI assistant. You are able to find answers to the questions from the contextual passage snippets provided.
"""
USER_PROMPT = f"""
Use the following pieces of information enclosed in <context> tags to provide an answer to the question enclosed in <question> tags.
<context>
{context}
</context>
<question>
{question}
</question>
"""

In [ ]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="qwen3:4b")

messages = [
    (
        "system",
        SYSTEM_PROMPT
    ),
    ("human", USER_PROMPT),
]
ai_msg = llm.invoke(messages)    
print(ai_msg.content)

<think>
Okay, the user is asking how data is stored in Milvus. Let me check the context provided.

From the first snippet, it says that Milvus stores two types of data: inserted data and metadata. Inserted data, like vectors, scalars, and schema, are stored in persistent storage as incremental logs. The supported backends include MinIO, S3, GCS, Azure, Alibaba OSS, and Tencent COS. Then metadata are stored in etcd.

Another part mentions that when data is inserted, it's loaded into a message queue. The data isn't flushed to disk until the flush() function is called, which forces writing to persistent storage. So the process involves message queues and then persistent storage.

So putting it all together, the answer should mention that inserted data is stored in persistent storage (with specific backends) as incremental logs, and metadata is stored in etcd. Also, the data goes through a message queue before being flushed to disk.
</think>

Milvus stores data in two forms: **inserted dat